In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

csv.field_size_limit(1000000000000000000)

131072

In [2]:
MASDEV = pd.read_pickle("./3_MASTER_DEVICE_File.pkl")

In [3]:
MASDEV.shape

(433946, 30)

In [4]:
# find columns to use for a manufacturer identifier and device identifier
masdevCols = []

for col in MASDEV:
    masdevCols.append(str(col))
    print(col)
len(masdevCols)

MDR_REPORT_KEY
REPORT_NUMBER
REPORT_SOURCE_CODE
MANUFACTURER_LINK_FLAG_
ADVERSE_EVENT_FLAG
PRODUCT_PROBLEM_FLAG
DATE_OF_EVENT
REPORTER_OCCUPATION_CODE
HEALTH_PROFESSIONAL
INITIAL_REPORT_TO_FDA
REPORT_TO_FDA
EVENT_LOCATION
EVENT_TYPE
TYPE_OF_REPORT
DATE_ADDED
DATE_CHANGED
REPORTER_COUNTRY_CODE
SUMMARY_REPORT
DEVICE_SEQUENCE_NO
DATE_RECEIVED
BRAND_NAME
GENERIC_NAME
MANUFACTURER_D_NAME
DEVICE_OPERATOR
MODEL_NUMBER
DEVICE_REPORT_PRODUCT_CODE
DEVICE_AGE_TEXT
DEVICE_EVALUATED_BY_MANUFACTUR
DEVICE_SEQUENCE_NO
year


30

In [5]:
# can be used for manufacturer identifier: MANUFACTURER_D_NAME
# can be used for product identifier: BRAND_NAME, GENERIC_NAME, MODEL_NUMBER

In [6]:
manufacCounts = MASDEV['MANUFACTURER_D_NAME'].value_counts(normalize=True).to_frame()
manufacCounts['count'] = MASDEV['MANUFACTURER_D_NAME'].value_counts()
manufacCounts.shape

(743, 2)

In [7]:
manufacCounts[manufacCounts['count'] >= 100].iloc[50:]

,MANUFACTURER_D_NAME,count
SORIN GROUP ITALIA S.R.L.,0.000378,164
OSCOR,0.000355,154
BIOTRONIK SE & CO KG,0.000346,150
ANGEION CORP.,0.000339,147
MEDTRONIC INC,0.000328,142
"SORIN BIOMEDICA C.R.M., S.R.L.",0.000318,138
"CARDIAC PACEMAKERS, INC.",0.000314,136
ST. JUDE MEDICAL INC. CRMD,0.000311,135
CLONMEL,0.000305,132
"ELA MEDICAL, S.A.S.",0.000279,121


In [8]:
# create  STD_MANUFACTURER
MASDEV["STD_MANUFACTURER"] = MASDEV["MANUFACTURER_D_NAME"]
MASDEV.head()

,MDR_REPORT_KEY,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,ADVERSE_EVENT_FLAG,PRODUCT_PROBLEM_FLAG,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,HEALTH_PROFESSIONAL,INITIAL_REPORT_TO_FDA,...,GENERIC_NAME,MANUFACTURER_D_NAME,DEVICE_OPERATOR,MODEL_NUMBER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,DEVICE_SEQUENCE_NO,year,STD_MANUFACTURER
0,257484,2938836-2000-00003,M,Y,Y,N,12/07/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-105C,LWS,3 YR,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"
1,257721,MW1017857,P,N,N,Y,12/10/1999,000,Y,NaN,...,IMPLANTABLE CARDIOVERTER DEFIBRILLATOR,ST. JUDE MEDICAL,0HP,V-175-C,LWS,NaN,NaN,1,2000,ST. JUDE MEDICAL
2,257784,257784,U,N,N,Y,12/07/1999,500,Y,Y,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"MEDTRONIC, INC.",0HP,7271,LWS,*,NaN,1,2000,"MEDTRONIC, INC."
3,257800,2938836-2000-00002,M,Y,Y,N,12/08/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-175C,LWS,9 MO,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"
4,257801,2938836-2000-00004,M,Y,Y,N,11/05/1999,001,Y,N,...,IMPLANTABLE CARDIAC DEFIBRILLATOR,"ST. JUDE MEDICAL, INC., CRMD",0HP,V-190HV3,LWS,2 YR,Y,1,2000,"ST. JUDE MEDICAL, INC., CRMD"


In [9]:
# create induvidual manufacturer dfs
# run twice to make sure all have the same length (for merging)

In [40]:
MASDEV["MPRI_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('MPRI', regex=False))
MASDEV_MPRI = MASDEV[MASDEV["MPRI_MADE"] == True]
MASDEV_MPRI = MASDEV_MPRI.assign(STD_MANUFACTURER='MEDTRONIC')
MASDEV_MPRI.shape

(46085, 47)

In [41]:
MASDEV["GUIDANT_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('GUIDANT', regex=False))
MASDEV_GUIDANT = MASDEV[MASDEV["GUIDANT_MADE"] == True]
MASDEV_GUIDANT = MASDEV_GUIDANT.assign(STD_MANUFACTURER='BOSTON SCIENTIFIC')
MASDEV_GUIDANT.shape

(44720, 47)

In [42]:
# FIX THIS
MASDEV["ST. JUDE_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('ST', regex=False) &
                                      MASDEV['MANUFACTURER_D_NAME'].str.contains('JUDE', regex=False))
MASDEV_StJUDE = MASDEV[MASDEV["ST. JUDE_MADE"] == True]
MASDEV_StJUDE = MASDEV_StJUDE.assign(STD_MANUFACTURER='ST. JUDE')
MASDEV_StJUDE.shape

(142433, 47)

In [43]:
MASDEV["CPI_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('CPI ', regex=False))
MASDEV_CPI = MASDEV[MASDEV["CPI_MADE"] == True]
MASDEV_CPI = MASDEV_CPI.assign(STD_MANUFACTURER='BOSTON SCIENTIFIC')
MASDEV_CPI.shape

(29526, 47)

In [44]:
MASDEV["MEDTRONIC_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC = MASDEV[MASDEV["MEDTRONIC_MADE"] == True]
MASDEV_MEDTRONIC = MASDEV_MEDTRONIC.assign(STD_MANUFACTURER='MEDTRONIC')
MASDEV_MEDTRONIC.shape

(90290, 47)

In [45]:
MASDEV["CARDIAC PACEMAKERS_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('CARDIAC PACEMAKERS', regex=False))
MASDEV_CARDIACPACEMAKERS = MASDEV[MASDEV["CARDIAC PACEMAKERS_MADE"] == True]
MASDEV_CARDIACPACEMAKERS = MASDEV_CARDIACPACEMAKERS.assign(STD_MANUFACTURER='BOSTON SCIENTIFIC')
MASDEV_CARDIACPACEMAKERS.shape

(26836, 47)

In [46]:
MASDEV["BOSTON SCIENTIFIC_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOSTONSCIENTIFIC = MASDEV[MASDEV["BOSTON SCIENTIFIC_MADE"] == True]
MASDEV_BOSTONSCIENTIFIC = MASDEV_BOSTONSCIENTIFIC.assign(STD_MANUFACTURER='BOSTON SCIENTIFIC')
MASDEV_BOSTONSCIENTIFIC.shape

(15960, 47)

In [47]:
MASDEV["BIOTRONIK_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK = MASDEV[MASDEV["BIOTRONIK_MADE"] == True]
MASDEV_BIOTRONIK = MASDEV_BIOTRONIK.assign(STD_MANUFACTURER='BIOTRONIK')
MASDEV_BIOTRONIK.shape

(17189, 47)

In [48]:
MASDEV["IPG MFG_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('IPG', regex=False))
MASDEV_IPGMFG = MASDEV[MASDEV["IPG MFG_MADE"] == True]
MASDEV_IPGMFG = MASDEV_IPGMFG.assign(STD_MANUFACTURER='IPG MFG')
MASDEV_IPGMFG.shape

(6111, 47)

In [49]:
MASDEV["ABBOTT_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('ABBOTT', regex=False))
MASDEV_ABBOTT = MASDEV[MASDEV["ABBOTT_MADE"] == True]
MASDEV_ABBOTT = MASDEV_ABBOTT.assign(STD_MANUFACTURER='ABBOTT')
MASDEV_ABBOTT.shape

(4560, 47)

In [50]:
MASDEV["CAMERON HEALTH_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('CAMERON HEALTH', regex=False))
MASDEV_CAMERONHEALTH = MASDEV[MASDEV["CAMERON HEALTH_MADE"] == True]
MASDEV_CAMERONHEALTH = MASDEV_CAMERONHEALTH.assign(STD_MANUFACTURER='CAMERON HEALTH')
MASDEV_CAMERONHEALTH.shape

(2541, 47)

In [51]:
MASDEV["SORIN_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('SORIN', regex=False))
MASDEV_SORIN = MASDEV[MASDEV["SORIN_MADE"] == True]
MASDEV_SORIN = MASDEV_SORIN.assign(STD_MANUFACTURER='SORIN')
MASDEV_SORIN.shape

(3579, 47)

In [52]:
MASDEV["ELA_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('ELA ', regex=False))
MASDEV_ELA = MASDEV[MASDEV["ELA_MADE"] == True]
MASDEV_ELA = MASDEV_ELA.assign(STD_MANUFACTURER='SORIN')
MASDEV_ELA.shape

(560, 47)

In [53]:
MASDEV["RICE CREEK_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('RICE CREEK', regex=False))
MASDEV_RICECREEK = MASDEV[MASDEV["RICE CREEK_MADE"] == True]
MASDEV_RICECREEK = MASDEV_RICECREEK.assign(STD_MANUFACTURER='MEDTRONIC')
MASDEV_RICECREEK.shape

(213, 47)

In [54]:
MASDEV["OSCOR_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('OSCOR', regex=False))
MASDEV_OSCOR = MASDEV[MASDEV["OSCOR_MADE"] == True]
MASDEV_OSCOR = MASDEV_OSCOR.assign(STD_MANUFACTURER='OSCOR')
MASDEV_OSCOR.shape

(246, 47)

In [55]:
MASDEV["ANGEION_MADE"] = pd.DataFrame(MASDEV['MANUFACTURER_D_NAME'].str.contains('ANGEION', regex=False))
MASDEV_ANGEION = MASDEV[MASDEV["ANGEION_MADE"] == True]
MASDEV_ANGEION = MASDEV_ANGEION.assign(STD_MANUFACTURER='ANGEION')
MASDEV_ANGEION.shape

(254, 47)

In [56]:
# append dfs together
manufacDFList = [ 
                MASDEV_GUIDANT, MASDEV_StJUDE, MASDEV_CPI, MASDEV_MEDTRONIC, MASDEV_CARDIACPACEMAKERS,
                MASDEV_BOSTONSCIENTIFIC, MASDEV_BIOTRONIK, MASDEV_IPGMFG, MASDEV_ABBOTT, MASDEV_CAMERONHEALTH,
                MASDEV_SORIN, MASDEV_ELA, MASDEV_RICECREEK, MASDEV_OSCOR, MASDEV_ANGEION
                ]

MASDEV2 = MASDEV_MPRI
print(MASDEV2.shape)

(46085, 47)


In [57]:
for df in manufacDFList:
    MASDEV2 = MASDEV2.append(df)
    print(MASDEV2.shape)
    #print(df.shape)

(90805, 47)
(233238, 47)
(262764, 47)
(353054, 47)
(379890, 47)
(395850, 47)
(413039, 47)
(419150, 47)
(423710, 47)
(426251, 47)
(429830, 47)
(430390, 47)
(430603, 47)
(430849, 47)
(431103, 47)


In [58]:
MASDEV2.shape

(431103, 47)

In [59]:
# drop duplicate report key values
s=MASDEV2['MDR_REPORT_KEY'].duplicated(keep='first')
indexDupeReportkey = s[s].index
print("there are " + str(len(indexDupeReportkey)) + " duplicate values in MASDEV2")

MASDEV2 = MASDEV2.drop_duplicates(subset=['MDR_REPORT_KEY'], keep='first')

s=MASDEV2['MDR_REPORT_KEY'].duplicated(keep='first')
indexDupeReportkey = s[s].index
print("there are now " + str(len(indexDupeReportkey)) + " duplicate values in MASDEV2 after removal")
print("the shape of MASDEV2 is now: " + str(MASDEV2.shape))

there are 245 duplicate values in MASDEV2
there are now 0 duplicate values in MASDEV2 after removal
the shape of MASDEV2 is now: (430858, 47)


In [60]:
# remove unwanted columns
print("the shape of MASDEV2 is: " + str(MASDEV2.shape))

keptCols = ['MDR_REPORT_KEY',
            'REPORT_NUMBER',
            'REPORT_SOURCE_CODE',
            'MANUFACTURER_LINK_FLAG_',
            'ADVERSE_EVENT_FLAG',
            'PRODUCT_PROBLEM_FLAG',
            'DATE_OF_EVENT',
            'REPORTER_OCCUPATION_CODE',
            'HEALTH_PROFESSIONAL',
            'INITIAL_REPORT_TO_FDA',
            'REPORT_TO_FDA',
            'EVENT_LOCATION',
            'EVENT_TYPE',
            'TYPE_OF_REPORT',
            'DATE_ADDED',
            'DATE_CHANGED',
            'REPORTER_COUNTRY_CODE',
            'SUMMARY_REPORT',
            'DEVICE_SEQUENCE_NO',
            'DATE_RECEIVED',
            'BRAND_NAME',
            'GENERIC_NAME',
            'DEVICE_OPERATOR',
            'MODEL_NUMBER',
            'DEVICE_REPORT_PRODUCT_CODE',
            'DEVICE_AGE_TEXT',
            'DEVICE_EVALUATED_BY_MANUFACTUR',
            'year',
            'STD_MANUFACTURER']

MASDEV2 = MASDEV2[keptCols]
print("the shape of MASDEV2 after column removal is: " + str(MASDEV2.shape))
print(MASDEV2.columns)

the shape of MASDEV2 is: (430858, 47)
the shape of MASDEV2 after column removal is: (430858, 30)
Index(['MDR_REPORT_KEY', 'REPORT_NUMBER', 'REPORT_SOURCE_CODE',
       'MANUFACTURER_LINK_FLAG_', 'ADVERSE_EVENT_FLAG', 'PRODUCT_PROBLEM_FLAG',
       'DATE_OF_EVENT', 'REPORTER_OCCUPATION_CODE', 'HEALTH_PROFESSIONAL',
       'INITIAL_REPORT_TO_FDA', 'REPORT_TO_FDA', 'EVENT_LOCATION',
       'EVENT_TYPE', 'TYPE_OF_REPORT', 'DATE_ADDED', 'DATE_CHANGED',
       'REPORTER_COUNTRY_CODE', 'SUMMARY_REPORT', 'DEVICE_SEQUENCE_NO',
       'DEVICE_SEQUENCE_NO', 'DATE_RECEIVED', 'BRAND_NAME', 'GENERIC_NAME',
       'DEVICE_OPERATOR', 'MODEL_NUMBER', 'DEVICE_REPORT_PRODUCT_CODE',
       'DEVICE_AGE_TEXT', 'DEVICE_EVALUATED_BY_MANUFACTUR', 'year',
       'STD_MANUFACTURER'],
      dtype='object')


In [61]:
MASDEV2.to_pickle("./3_MASTER_DEVICE_File_wSTDMANUFAC.pkl")